In [1]:
import pandas as pd
import string
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter
import re
%matplotlib inline

In [ ]:
find_largest_classes[patent_class_citation, 'Class']

### Observations

#### class.csv:
- There are 4,906,289 unique patents
- data contains patent number, USPC class and subclass, designation of whether class is primary class
- some patent numbers read in as string, some as int, may be duplicates
- 19.8 M rows, multiple classes for a given patent
- 200,000 patents do not have a primary class
- 37,173 patents with multiple primary classes
- No missing values

#### citation.csv:
- There are 4,130,388 unique patents
- 53,527,305 total citations ==> roughly 13 citations per patent
- data contains patent number, publication date of cited patent, last name of frst author of cited patent, `Cit_Kind`, `Cit_Country`, Number of patent cited (`Citation`), `Category`,  CitSeq'
- More than half of `Cit_kind`, `Cit_Country` and `Category` are missing
- No patent or citation missing values

#### citation.csv:
- Both `Kind` and `pattype` contain too many missing values to identify utility patents
- 4,821,608 unique patents, 4,823,407 total patents
- Stripping out patents that are not formatted as utility patents leaves 4,511,191 patents

Note on how utility patents were identified: The format of patent numbers indicate their type.

[Types of patents](https://www.uspto.gov/patents-application-process/applying-online/patent-number):
- Utility : Patent numbers consist of six or seven digits. Enter the Patent Number excluding commas and spaces and omit leading zeroes. 
- Reissue : (e.g., Rennnnnn, RE000126) must enter leading zeroes between "RE" and number to create 6 digits. 
- Plant Patents :(e.g., PPnnnnnn, PP000126) must enter leading zeroes between "PP" and number to create 6 digits. 
- Design : (e.g., Dnnnnnnn, D0000126) must enter leading zeroes between "D" and number to create 7 digits. 
- X Patents : (e.g., Xnnnnnnn , X0000001) must enter leading zeroes between "X" and number to create 7 digits. 
- H Documents : (e.g., Hnnnnnnn , H0000001) must enter leading zeroes between "H" and number to create 7 digits. 
- T Documents : (e.g., Tnnnnnnn , T0000001) must enter leading zeroes between "T" and number to create 7 digits. 

In [ ]:
patent['Utility'] = patent['Patent'].apply(is_utility)

In [2]:
def find_largest_classes(df, col):
    '''Return num rows for each '''
    sizes = []
    for c in set(df[col]):
        sizes.append((c, df[df[col] == c].shape[0]))
    return sizes

In [3]:
def strip_patents_multiple_primary_classes(df, col):
    '''Return modified data frame without rows that had duplicate
    value for given column'''
    return df[col].duplicated()

In [4]:
def percentage_nans(df, rounding=False):
    '''Return the percentage of missing values for each column of a pandas df'''
    nan_share = df.isnull().sum(axis=0) * 1.0 / df.shape[0]
    if rounding:
        return nan_share.round(rounding)
    else:
        return nan_share

In [5]:
def clean_class(class_str):
    '''Convert string USPTO classification number to int if number
    class does not begin with letter'''
    if class_str[0] not in set(string.ascii_uppercase): 
        return int(class_str) 

In [6]:
def is_utility(patent_num):
    '''Returns 1 if patent_num string is formatted
    consistent with reissue, plant, design patent or X, H or T doc type'''
    if type(patent_num) != str:
        return 1
    return re.findall('RE|PP|D|X|H|T', patent_num) == []

#### Data prep

In [ ]:
patclass = pd.read_csv('class.csv')
citation = pd.read_csv('citation.csv')
patent = pd.read_csv('patent.csv')


#extract columns info from citation data w/ complete data
citation_clean = citation[['Patent', 'Citation']]


## PROCESS CLASS FILE
#standardize class
patclass['Class'] = patclass['Class'].apply(clean_class)
#kept just 'Primary' class
primary_class = patclass[patclass.Prim == 1]
#drop patents that had multiple primary classes
unique_patent_primary_class = primary_class[primary_class.Patent.duplicated() == False]
class_clean = unique_patent_primary_class[['Patent', 'Class', 'SubClass']]
class_clean.columns = ['Patent', 'primary_class', 'primary_subclass']


## Patent file preparation
patent['Utility'] = patent['Patent'].apply(is_utility)
patent_utility = patent[patent.Utility == 1]
patent_clean = utility_patent[['Patent', 'GDate', 'GYear']]


# MERGE THE DFs TOGETHER
_ = pd.merge(class_clean, citation_clean, on='Patent')
patent_class_citation = pd.merge(_, patent_clean, on='Patent')

/Users/zoerichards/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
